# <font color = 'pickle'> **Bag of Words (Sparse Embeddings)**


<font color = 'pickle' size =5 >**What is Bag of Words (BoW)?**</font>

A **bag-of-words** is a representation of text that describes the occurrence of words within a document <font color ='indianred'>**disregarding grammar and word order**</font>. It involves two steps:

    1. Create Vocabulary. Each word in vocabulary forms feature(independent variable) to represent document.
    2. Score words (based on frequency or occurrence) to create Vectors.

<font color = 'pickle' size =5> **Why do you need to learn Bag of Words?**</font>

- Till now we have learnt how to pre-process the text data i.e clean the text data.
- Our final goal is to use text data in Machine Learning (ML) models. For example - we want to predict whether e-mail is a spam or not based on the text of the data.
- But ML models can understand only numbers. Therefore we need to convert text to vectors (numbers).
- The simple method of converting text to numbers is to use 'Bag of Words approach'






<font color = 'pickle' size =5> **Why do you need Bag of Words in age of LLMs?** </font>

<font color = 'indianred'> **Outstanding paper Award ACL 2023: Linear Classifier: An Often-Forgotten Baseline for Text Classification**</font>

*Large-scale pre-trained language models such as BERT are popular solutions for text classification. Due to the superior performance of these advanced methods, nowadays, people often directly train them for a few epochs and deploy the obtained model. In this opinion paper, we point out that this way may only sometimes get satisfactory results. We argue the importance of running a simple baseline like linear classifiers on bag-of-words features along with advanced methods.*



## <font color = 'pickle' size =5 >**Learning Outcome** </font>
After completing this tutorial, you will know

1. What the bag-of-words approach is and how you can use it to represent text data.
2. What are different techniques to prepare a vocabulary and score words.
3. How to implement 'Bag-of-words' approach in python using sklearn.

# <font color = 'pickle'> **Tutorial Overview**</font>
 - Generating Vocab
 - Generating vectors using Vocab
     - Binary Vectorizer
     - Count Vectorizer
     - tfidf Vectorizer

 - Modifying Vocab
 - Example - IMDB Dataset


# <font color = 'pickle'> Import/install Libraries

In [1]:
%load_ext autoreload
%autoreload 2

The code `%load_ext autoreload` and `%autoreload 2` in a Jupyter notebook enables the autoreload extension. This setup automatically reloads imported modules before executing code cells. Specifically, `%autoreload 2` ensures all modules are reloaded each time before execution, reflecting any changes made to the module files without needing to restart the notebook kernel.

In [2]:
import sys
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    !pip install -U nltk -qq
    !pip install -U spacy -qq
    !pip install -U scikit-optimize -qq
    !python -m spacy download en_core_web_sm -qq
    !pip install pyspellchecker -qq
    !pip install optuna -qq

    basepath = '/content/drive/MyDrive/Colab_Notebooks/BUAN_6342_Applied_Natural_Language_Processing'
    sys.path.append('/content/drive/MyDrive/Colab_Notebooks/BUAN_6342_Applied_Natural_Language_Processing/0_Custom_files')
else:
    basepath = '/Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6342_Applied_Natural_Language_Processing/'
    sys.path.append(
        '/Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6342_Applied_Natural_Language_Processing/0_Custom_files')

**Code Explanation**

1. **Environment Detection**:
The code begins by checking the environment in which it's running. This is done using the statement `if 'google.colab' in str(get_ipython()):`. If this condition is true, it means the code is running in Google Colab, an online interactive development environment. If not, the code assumes it's running in a local environment (like your personal computer).

2. **Setup for Google Colab:**
When running in Google Colab, the code performs the following actions:

- *Mount Google Drive*: `from google.colab import drive` and `drive.mount('/content/drive')` are used to mount the user's Google Drive to the Colab environment, enabling access to files stored there.

- *Install Necessary Libraries*: The code installs or updates specific Python libraries (`nltk` and `spacy`) silently without producing unnecessary output (`-qq`).

- *Download spaCy Model*: `!python -m spacy download en_core_web_sm -qq` downloads the English language model for spaCy, necessary for NLP tasks.

- *Set Basepath and Update sys.path*:
  - `basepath` is set to a path in Google Drive where data or relevant files are stored.
  - `sys.path.append('/content/drive/MyDrive/data/custom-functions')` adds a directory to the Python search path. This allows Python to import and use custom functions located in that directory in the Colab notebook, promoting modularity and code reusability.

3. **Setup for Local Environment:**
If the code is not running in Google Colab, it's presumed to be in a local environment. Here, the setup is slightly different:

- *Set Basepath*: `basepath` is set to a specific directory on the local machine. This is where the code will look for data files or other resources.

- *Update sys.path for Custom Functions*:
  - `sys.path.append('/home/harpreet/Insync/google_drive_shaannoor/data/custom-functions')` adds a local directory to the Python search path. Similar to the Colab setup, this step allows the local Python environment to import and use custom functions from the specified directory.

In [3]:
# Import required libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations
import spacy         # For NLP preprocessing

# Import required nltk packages
import nltk
nltk.download('stopwords')  # Download the stopwords corpus
from nltk.corpus import stopwords as nltk_stopwords  # Stopwords corpus

# Import tweet tokenizer from nltk
from nltk.tokenize import TweetTokenizer

# Import CountVectorizer and TfidfVectorizer from scikit-learn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Import pathlib for managing file paths
from pathlib import Path

# import custom-preprocessor from python file
import CustomPreprocessorSpacy as cp

/var/folders/3d/fk1zpy415g31bg07yrkc5qbm0000gn/T/ipykernel_17072/837683949.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd  # For data manipulation and analysis
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harikrishnadev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
spacy.__version__


'3.7.2'

In [5]:
# load spacy model
nlp = spacy.load('en_core_web_sm')


## <font color = 'pickle'> **Generating Vocab**

###  <font color = 'pickle'> **Dummy Corpus**

In [6]:
# Dummy corpus
Corpus = ["Count Vectorizer - for this vectorizer, scoring is done based on frequency. For this vectorizer frequency is key. @vectorizer #frequency @frequency, doesn’t",
          "tfidf vectorizer - for this vectorizer, scoring is done based on tfidf,  higher tfidf higher score #tfidf @vectorizer "  ,
          "Binary vectorizer - for this vectorizer, scoring is done based on presence of word. For this vectorizer, dummy is key #dummy @dummy @vectorizer "]


### <font color = 'pickle'>**Create an instance of Vectorizer**

In [7]:
vectorizer = CountVectorizer()


The above code creates an instance of the `CountVectorizer` class from the `sklearn.feature_extraction.text module`. This class is used to convert a collection of text documents to a matrix of token counts.

It accomplishes this by
  1. tokenizing the input text
  2. creating a vocabulary of all the tokens found in the text
  3. encoding the text as a matrix of token counts based on this vocabulary.

The created instance vectorizer can then be used to fit the text data to the vocabulary and generate the token count matrix.

In [8]:
CountVectorizer??


Init signature:
CountVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.int64'>,
)
Source:        
class CountVectorizer(_VectorizerMixin, BaseEstimator):
    r"""Convert a collection of text documents to a matrix of token counts.

    This implementation produces a sparse representation of the counts using
    scipy.sparse.csr_matrix.

    If you do not provide an a-priori dictionary and you do not use an analyzer
    that does some kind of feature selection then the number of features will
    be equal to the vocabulary size found by analyzing the data.

    For an efficiency comparison of the different feature extractors, see
    :ref:`sphx_gl

### <font color = 'pickle'>**Fit Vectorizer on corpus to generate vocab**

In [9]:
# Fit the vectorizer on corpus
vectorizer.fit(Corpus)


CountVectorizer()

<font color = 'indianred'>**Vectorizer().fit() does the following**:
- lowercases your text
- uses utf-8 encoding
- performs tokenization (converts raw text to smaller units of text)
- uses word level tokenization (meaning each word is treated as a separate token) and  ignores single characters during tokenization ( words like ‘a’ and ‘I’ are removed)
- By default, the regular expression that is used to split the text and create tokens is : `"\b\w\w+\b"`.
  - This means it finds all sequences of characters that consist of at least two letters or numbers(\w) and that are separated by word boundaries (\b).
  - It does not find single-letter words, and it splits up contractions like “doesn’t” or “bit.ly”, but it matches “h8ter” as a single word.
- The CountVectorizer then converts all words to lowercasecharacters, so that “soon”, “Soon”, and “sOon” all correspond to the same token (and therefore feature).
- It then creates a dictionary of unique words.
- The set of unique words is used as features in the CountVectorizer.

In [10]:
# Display the mapping of terms to feature indices created by the vectorizer
vectorizer.vocabulary_


{'count': 2,
 'vectorizer': 18,
 'for': 6,
 'this': 17,
 'scoring': 15,
 'is': 9,
 'done': 4,
 'based': 0,
 'on': 12,
 'frequency': 7,
 'key': 10,
 'doesn': 3,
 'tfidf': 16,
 'higher': 8,
 'score': 14,
 'binary': 1,
 'presence': 13,
 'of': 11,
 'word': 19,
 'dummy': 5}

We can use `vectorizer.get_feature_names_out()` to obtain the list of unique words that the CountVectorizer has identified from the corpus

In [11]:
# Retrieve the list of unique words (tokens) that the CountVectorizer identified in the corpus.
# These words serve as features (columns) in the vectorized output.
features = vectorizer.get_feature_names_out()
print(features)

# Print the total number of unique features (words) identified in the corpus
print('\ntotal number of unique features (words):', len(features))




['based' 'binary' 'count' 'doesn' 'done' 'dummy' 'for' 'frequency'
 'higher' 'is' 'key' 'of' 'on' 'presence' 'score' 'scoring' 'tfidf' 'this'
 'vectorizer' 'word']

total number of unique features (words): 20


## <font color = 'pickle'>**Generate Vectors using Vocab**

### <font color = 'pickle'>**Binary Vectorizer**

In [12]:
binary_vectorizer = CountVectorizer(binary=True)
binary_vectorizer.fit(Corpus)


CountVectorizer(binary=True)

- We can now call transform() method to transform documents in our corpus to vectors.
- <font color = 'dodgerblue'>**Each document**</font> will be represented by <font color = 'dodgerblue'>**vector of length equal to len(dictionary)**.</font>
- The vectors are stored in the form of a <font color = 'dodgerblue'>**sparse matrix**.</font>
- We can use <font color = 'dodgerblue'>**toarray()**</font> function to get complete matrix.
- Number of columns represent the number of features (len(vocab)).
- Number of rows represent the number the documents in a corpus.
- <font color = 'dodgerblue'>**For each row, the numbers displayed are 0 or 1 - indicating absence or presence of a word in a document.**

In [13]:
binary_vectorizer.vocabulary_

{'count': 2,
 'vectorizer': 18,
 'for': 6,
 'this': 17,
 'scoring': 15,
 'is': 9,
 'done': 4,
 'based': 0,
 'on': 12,
 'frequency': 7,
 'key': 10,
 'doesn': 3,
 'tfidf': 16,
 'higher': 8,
 'score': 14,
 'binary': 1,
 'presence': 13,
 'of': 11,
 'word': 19,
 'dummy': 5}

In [14]:
binary_vectors = binary_vectorizer.transform(Corpus)

In [15]:
print(f'vectors in sparse format')
print(binary_vectors)


vectors in sparse format
  (0, 0)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 6)	1
  (0, 7)	1
  (0, 9)	1
  (0, 10)	1
  (0, 12)	1
  (0, 15)	1
  (0, 17)	1
  (0, 18)	1
  (1, 0)	1
  (1, 4)	1
  (1, 6)	1
  (1, 8)	1
  (1, 9)	1
  (1, 12)	1
  (1, 14)	1
  (1, 15)	1
  (1, 16)	1
  (1, 17)	1
  (1, 18)	1
  (2, 0)	1
  (2, 1)	1
  (2, 4)	1
  (2, 5)	1
  (2, 6)	1
  (2, 9)	1
  (2, 10)	1
  (2, 11)	1
  (2, 12)	1
  (2, 13)	1
  (2, 15)	1
  (2, 17)	1
  (2, 18)	1
  (2, 19)	1


In [16]:
print(f'\nbinary vectors in array(dense) format')
print(binary_vectors.toarray())
print(
    f'\nThe shape of the binary vectors is : {binary_vectors.toarray().shape}')



binary vectors in array(dense) format
[[1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 1 0 1 1 0]
 [1 0 0 0 1 0 1 0 1 1 0 0 1 0 1 1 1 1 1 0]
 [1 1 0 0 1 1 1 0 0 1 1 1 1 1 0 1 0 1 1 1]]

The shape of the binary vectors is : (3, 20)


In [17]:
# create dataframe for better visualization
df_binary = pd.DataFrame(binary_vectors.toarray(), columns=features)
df_binary


,based,binary,count,doesn,done,dummy,for,frequency,higher,is,key,of,on,presence,score,scoring,tfidf,this,vectorizer,word
0,1,0,1,1,1,0,1,1,0,1,1,0,1,0,0,1,0,1,1,0
1,1,0,0,0,1,0,1,0,1,1,0,0,1,0,1,1,1,1,1,0
2,1,1,0,0,1,1,1,0,0,1,1,1,1,1,0,1,0,1,1,1


### <font color = 'pickle'>**Count Vectorizer**
-  The vectors are stored in the form of a sparse matrix.
- Number of columns represent the number of features (len(vocab))
- Number of rows represent the number the documents in a corpus
- Thus, each document is represented by a vector of size of length of vocab.
- For each row, <font color = 'dodgerblue'>**the numbers displayed are the number of times a particular word has occurred in the document.**

In [18]:
term_freq_vectorizer = CountVectorizer(binary=False)
# we can combine fit and transform steps into a single step using fit_transform()
count_vectors = term_freq_vectorizer.fit_transform(Corpus)
print(f'count vectors in array (dense) format\n')
print(count_vectors.toarray())
print(f'\nThe shape of the count vectors is : {count_vectors.toarray().shape}')


count vectors in array (dense) format

[[1 0 1 1 1 0 2 4 0 2 1 0 1 0 0 1 0 2 4 0]
 [1 0 0 0 1 0 1 0 2 1 0 0 1 0 1 1 4 1 3 0]
 [1 1 0 0 1 3 2 0 0 2 1 1 1 1 0 1 0 2 4 1]]

The shape of the count vectors is : (3, 20)


In [19]:
# create dataframe for better visualization
df_count = pd.DataFrame(count_vectors.toarray(),
                        columns=term_freq_vectorizer.get_feature_names_out())
df_count


,based,binary,count,doesn,done,dummy,for,frequency,higher,is,key,of,on,presence,score,scoring,tfidf,this,vectorizer,word
0,1,0,1,1,1,0,2,4,0,2,1,0,1,0,0,1,0,2,4,0
1,1,0,0,0,1,0,1,0,2,1,0,0,1,0,1,1,4,1,3,0
2,1,1,0,0,1,3,2,0,0,2,1,1,1,1,0,1,0,2,4,1


### <font color = 'pickle'>**tf-idf Vectorizer**</font>

- One measure of how important a word is term frequency (tf) (how frequently a word occurs in a document). We examined term frequency in previous sections where we used CountVectorizer to get the freqency of each word.
- But there may be words in a document, that occur many times but these words also occur in all other documents as well.
- Therefore the word might not be a good representation of the document.
- We can account for this by  <font color = 'dodgerblue'>giving more importance to words that occur in fewer documents using inverse document frequency </font> ((# Number of documents) / (Number of documents containing the word)).
- This can be <font color = 'dodgerblue'>combined with term frequency</font> to calculate a term’s tf-idf (the two quantities multiplied together), the frequency of a term adjusted for how rarely it is used.
- The idea of tf-idf is to <font color = 'dodgerblue'>find the important words for the content of each document by decreasing the weight for commonly used words and increasing the weight for words that are not used very much in a collection or corpus of documents.</font>
- tf-idf gives more weight to the the words that are important (i.e., occur more frequently) in a given document, but occur rarely in other documents.

In [20]:
tfidf_vectorizer = TfidfVectorizer()
# we can combine fit and transform steps into a single step using fit_transform()
tfidf_vectors = tfidf_vectorizer.fit_transform(Corpus)
print(f'tfidf vectors in array (dense) format\n')
print(tfidf_vectors.toarray())
print(f'\nThe shape of the tfidf vectors is : {tfidf_vectors.toarray().shape}')


tfidf vectors in array (dense) format

[[0.10829999 0.         0.18336782 0.18336782 0.10829999 0.
  0.21659998 0.73347128 0.         0.21659998 0.13945595 0.
  0.10829999 0.         0.         0.10829999 0.         0.21659998
  0.43319995 0.        ]
 [0.11455596 0.         0.         0.         0.11455596 0.
  0.11455596 0.         0.3879202  0.11455596 0.         0.
  0.11455596 0.         0.1939601  0.11455596 0.77584039 0.11455596
  0.34366788 0.        ]
 [0.11874019 0.20104462 0.         0.         0.11874019 0.60313387
  0.23748039 0.         0.         0.23748039 0.15289962 0.20104462
  0.11874019 0.20104462 0.         0.11874019 0.         0.23748039
  0.47496077 0.20104462]]

The shape of the tfidf vectors is : (3, 20)


In [21]:
# create dataframe for better visualization
df_tfidf = pd.DataFrame(tfidf_vectors.toarray(),
                        columns=tfidf_vectorizer.get_feature_names_out())
df_tfidf.round(4)


,based,binary,count,doesn,done,dummy,for,frequency,higher,is,key,of,on,presence,score,scoring,tfidf,this,vectorizer,word
0,0.1083,0.000,0.1834,0.1834,0.1083,0.0000,0.2166,0.7335,0.0000,0.2166,0.1395,0.000,0.1083,0.000,0.000,0.1083,0.0000,0.2166,0.4332,0.000
1,0.1146,0.000,0.0000,0.0000,0.1146,0.0000,0.1146,0.0000,0.3879,0.1146,0.0000,0.000,0.1146,0.000,0.194,0.1146,0.7758,0.1146,0.3437,0.000
2,0.1187,0.201,0.0000,0.0000,0.1187,0.6031,0.2375,0.0000,0.0000,0.2375,0.1529,0.201,0.1187,0.201,0.000,0.1187,0.0000,0.2375,0.4750,0.201


### <font color = 'pickle'>**Undertstanding tfidf calculations**

By default <br>
$\text{tfidf}(w, d) = \text{tf(w, d)} * \text{idf(w)}$
<br>
$\text{idf(w)} = \log\big(\frac{N + 1}{N_w + 1}\big) + 1$
<br><br>
if smooth_idf = False (default is True):
<br>
$\text{idf(w)} = \log\big(\frac{N }{N_w}\big) + 1$
<br><br>
if sublinear_tfbool = True (default is False)
<br>
$\text{tf(w, d)} = \log(\text{tf(w, d)} ) + 1$

Here:<br>
- $\text{tf}(w, d)$ is number of times word $w$ appears in document $d$
<br>
- $\text{idf}(w)$ is inverse document frequency of word $w$
- $N$ is total number of documents
- $N_w$ is number of documents that contain word w

In [22]:
# Calculate inverse document frequency for each feature (word)
term_idf = tfidf_vectorizer.idf_
term_idf


array([1.        , 1.69314718, 1.69314718, 1.69314718, 1.        ,
       1.69314718, 1.        , 1.69314718, 1.69314718, 1.        ,
       1.28768207, 1.69314718, 1.        , 1.69314718, 1.69314718,
       1.        , 1.69314718, 1.        , 1.        , 1.69314718])

In [23]:
# create dataframe for better visualization
df_idf = pd.DataFrame(term_idf, index=tfidf_vectorizer.get_feature_names_out())
df_idf.round(4).T

,based,binary,count,doesn,done,dummy,for,frequency,higher,is,key,of,on,presence,score,scoring,tfidf,this,vectorizer,word
0,1.0,1.6931,1.6931,1.6931,1.0,1.6931,1.0,1.6931,1.6931,1.0,1.2877,1.6931,1.0,1.6931,1.6931,1.0,1.6931,1.0,1.0,1.6931


For better understanding, we will now just look at the first document and get the idf and term frequencies for each word in the first document

In [24]:
# create dataframe for tf vectors for the first document

# Create a dense numpy array from the sparse count vector for the first document
first_document_tf = count_vectors[0].toarray().ravel()

# Get the feature names for the term frequency vectors
feature_names_tf = term_freq_vectorizer.get_feature_names_out()

# Create a dataframe from the term frequency feature names and values
df_tf = pd.DataFrame({'features': feature_names_tf, 'tf': first_document_tf})
df_tf


,features,tf
0,based,1
1,binary,0
2,count,1
3,doesn,1
4,done,1
5,dummy,0
6,for,2
7,frequency,4
8,higher,0
9,is,2


Note: The `toarray` method is used to convert the sparse matrix into a dense numpy array, and `ravel` is used to flatten the resulting 2-dimensional array into a 1-dimensional array. This is necessary because pandas dataframes expect 1-dimensional arrays as values for the columns.

In [25]:
# create dataframe for tfidf vectors for the first document
first_document_tfidf = tfidf_vectors[0].toarray().ravel()
feature_names_tfidf = tfidf_vectorizer.get_feature_names_out()
df_tfidf = pd.DataFrame({'features': feature_names_tfidf,
                        'idf': term_idf, 'norm_tfidf': first_document_tfidf})

# combine dataframes

# Merge the tf and tf-idf dataframes on the 'features' column
df = pd.merge(left=df_tf, right=df_tfidf)

# Sort the combined dataframe by the 'norm_tfidf' column in descending order
df.sort_values(by=["norm_tfidf"], ascending=False, inplace=True)

df


,features,tf,idf,norm_tfidf
7,frequency,4,1.693147,0.733471
18,vectorizer,4,1.000000,0.433200
17,this,2,1.000000,0.216600
6,for,2,1.000000,0.216600
9,is,2,1.000000,0.216600
2,count,1,1.693147,0.183368
3,doesn,1,1.693147,0.183368
10,key,1,1.287682,0.139456
12,on,1,1.000000,0.108300
15,scoring,1,1.000000,0.108300


**Observations from above results**
- words 'frequency' and 'vectorizer' occurs 4 times in the documsnt and hence term frequency is 4.
- Word 'vectorizer' occurs in every document and hence idf is 1 (log(1) + 1).
- norm_tfidf gives higher score to word 'frequency' than 'vectorizer'.
- norm_tfidf is not equal to idf * tf

Let us know understand how norm_tfidf is calculated:

In [26]:
# calculate tfidf (without any normalization)
df['tfidf'] = df.eval('tf*idf')


In [27]:
# calculate tfidf - normalized
df['sq_tfidf'] = df.eval('tfidf**2')
df['norm_tfidf_manually'] = df['tfidf']/np.sqrt(df['sq_tfidf'].sum())

In [28]:
df

,features,tf,idf,norm_tfidf,tfidf,sq_tfidf,norm_tfidf_manually
7,frequency,4,1.693147,0.733471,6.772589,45.867958,0.733471
18,vectorizer,4,1.000000,0.433200,4.000000,16.000000,0.433200
17,this,2,1.000000,0.216600,2.000000,4.000000,0.216600
6,for,2,1.000000,0.216600,2.000000,4.000000,0.216600
9,is,2,1.000000,0.216600,2.000000,4.000000,0.216600
2,count,1,1.693147,0.183368,1.693147,2.866747,0.183368
3,doesn,1,1.693147,0.183368,1.693147,2.866747,0.183368
10,key,1,1.287682,0.139456,1.287682,1.658125,0.139456
12,on,1,1.000000,0.108300,1.000000,1.000000,0.108300
15,scoring,1,1.000000,0.108300,1.000000,1.000000,0.108300


## <font color = 'pickle'>**Modifying Vocab**

### <font color = 'pickle'>**Case sensitive**

In [29]:
# The lowercase argument is set to False to indicate that the text should
# not be converted to lowercase before tokenizing.
# The resulting vocab may have same word in upper and lower case
vectorizer = CountVectorizer(lowercase=False)

# we can use fit_transform to use fit() and transform() in one step
vectors = vectorizer.fit_transform(Corpus)
vectorizer.vocabulary_


{'Count': 1,
 'Vectorizer': 3,
 'for': 8,
 'this': 19,
 'vectorizer': 20,
 'scoring': 17,
 'is': 11,
 'done': 6,
 'based': 4,
 'on': 14,
 'frequency': 9,
 'For': 2,
 'key': 12,
 'doesn': 5,
 'tfidf': 18,
 'higher': 10,
 'score': 16,
 'Binary': 0,
 'presence': 15,
 'of': 13,
 'word': 21,
 'dummy': 7}

### <font color = 'pickle'>**Filtering words based on frequency**

The `max_df`, `min_df`, and `max_features` parameters in the `CountVectorizer`` class control the feature selection for the resulting term frequency (tf) vectors.

- `max_df`: This parameter sets the maximum threshold for the frequency of a term in the document collection. If a term has a document frequency (i.e., the number of documents that contain the term) higher than max_df, it will be ignored. <font color = 'dodgerblue' >**This parameter is used to filter out stop words (corpus specific) that appear in too many documents.** </font>

- min_df: This parameter sets the minimum threshold for the frequency of a term in the document collection. If a term has a document frequency lower than min_df, it will be ignored.  <font color = 'dodgerblue' >**This parameter is used to filter out rare words that appear in too few documents.**

- max_features: This parameter sets the maximum number of features (i.e., the maximum number of unique terms) that should be included in the resulting tf vectors. If the number of unique terms in the document collection is larger than max_features, the terms with the highest tf values will be kept and the others will be ignored.  <font color = 'dodgerblue' >**This parameter is used to reduce the dimensionality of the resulting tf vectors, which can help reduce the computational cost of downstream processing.**

By using the max_df, min_df, and max_features parameters, you can control the feature selection process and determine the most informative terms to include in the tf vectors.

In [30]:
# remove rare words - remove words which appear in less than 2 documents
vectorizer = CountVectorizer(min_df=2)
vectorizer.fit(Corpus)
vectorizer.vocabulary_


{'vectorizer': 8,
 'for': 2,
 'this': 7,
 'scoring': 6,
 'is': 3,
 'done': 1,
 'based': 0,
 'on': 5,
 'key': 4}

In [31]:
# remove words which appear in more than 2 documents - remove corpus specific stop words
vectorizer = CountVectorizer(max_df=2)
vectorizer.fit(Corpus)
vectorizer.vocabulary_


{'count': 1,
 'frequency': 4,
 'key': 6,
 'doesn': 2,
 'tfidf': 10,
 'higher': 5,
 'score': 9,
 'binary': 0,
 'presence': 8,
 'of': 7,
 'word': 11,
 'dummy': 3}

In [32]:
# retain most frequent words only - retain top n words based on term frequency across corpus
vectorizer = CountVectorizer(max_features=5)
vectorizer.fit(Corpus)
vectorizer.vocabulary_


{'vectorizer': 4, 'for': 0, 'this': 3, 'is': 1, 'tfidf': 2}

### <font color = 'pickle'>**Stop Words**

In [33]:
# We can also specify list of stopwords to countvectorizer to get the feature without stopwords

# Import libraries
nltk_stop_words = nltk_stopwords.words('english')

vectorizer = CountVectorizer(max_features=5, stop_words=nltk_stop_words)
vectorizer.fit(Corpus)
vectorizer.vocabulary_


{'vectorizer': 4, 'done': 1, 'based': 0, 'frequency': 2, 'tfidf': 3}

### <font color = 'pickle'>**Custom Tokenizer and Preprocessor**

#### <font color = 'pickle'>**nltk tokenizer**

In [34]:
# We can use custom tokenizer e.g. we can use nltk tweet tokenizer to get each tokens as feature

# Create an instance of the TweetTokenizer class
tweet_tokenizer = TweetTokenizer()

# Initialize the CountVectorizer with the custom tokenizer
# only works if analyzer = 'word'
vectorizer = CountVectorizer(
    analyzer='word', tokenizer=tweet_tokenizer.tokenize)

vectorizer.fit_transform(Corpus)
vectorizer.vocabulary_

/opt/homebrew/anaconda3/envs/nlp/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'count': 11,
 'vectorizer': 28,
 '-': 4,
 'for': 15,
 'this': 27,
 ',': 3,
 'scoring': 24,
 'is': 18,
 'done': 13,
 'based': 9,
 'on': 21,
 'frequency': 16,
 '.': 5,
 'key': 19,
 '@vectorizer': 8,
 '#frequency': 1,
 '@frequency': 7,
 'doesn': 12,
 '’': 30,
 't': 25,
 'tfidf': 26,
 'higher': 17,
 'score': 23,
 '#tfidf': 2,
 'binary': 10,
 'presence': 22,
 'of': 20,
 'word': 29,
 'dummy': 14,
 '#dummy': 0,
 '@dummy': 6}

#### <font color = 'pickle'>**spacy pre-processor and tokenizer**

In [35]:
def spacy_preprocessor(text):

    # Create spacy object
    doc = nlp(text)

    # remove punctuations and get a list of tokens
    filtered_text = [token.text for token in doc if not token.is_punct]

    # join the processed tokens in to string
    return " ".join(filtered_text)


In [36]:
# Spacy Tokenizer
def spacy_tokenizer(data):
    doc = nlp(data)
    return [token.text for token in doc]


In [37]:
# custom preprocessor and spacy tokenizer
vectorizer = CountVectorizer(analyzer='word', preprocessor=spacy_preprocessor,
                             tokenizer=spacy_tokenizer, token_pattern=None)
vectors = vectorizer.fit(Corpus)
vectorizer.vocabulary_


{'Count': 5,
 'Vectorizer': 7,
 'for': 12,
 'this': 24,
 'vectorizer': 25,
 'scoring': 22,
 'is': 15,
 'done': 10,
 'based': 8,
 'on': 19,
 'frequency': 13,
 'For': 6,
 'key': 16,
 '@vectorizer': 3,
 '@frequency': 2,
 'does': 9,
 'n’t': 17,
 'tfidf': 23,
 '  ': 0,
 'higher': 14,
 'score': 21,
 'Binary': 4,
 'presence': 20,
 'of': 18,
 'word': 26,
 'dummy': 11,
 '@dummy': 1}

#### <font color = 'pickle'>**custom preprocessor we created earlier**

In [38]:
cp.SpacyPreprocessor??

Init signature:
cp.SpacyPreprocessor(
    model,
    *,
    batch_size=64,
    lemmatize=True,
    lower=True,
    remove_stop=True,
    remove_punct=True,
    remove_email=True,
    remove_url=True,
    remove_num=False,
    stemming=False,
    add_user_mention_prefix=True,
    remove_hashtag_prefix=False,
    basic_clean_only=False,
)
Docstring:     
A text preprocessor that utilizes spaCy for efficient and flexible NLP. Designed as a part of a scikit-learn
pipeline, it provides a wide range of text cleaning and preprocessing functionalities.

Attributes:
    model (str): The spaCy language model to be used for tokenization and other NLP tasks.
    batch_size (int): The number of documents to process at once during spaCy's pipeline processing.
    lemmatize (bool): If True, lemmatize tokens.
    lower (bool): If True, convert all characters to lowercase.
    remove_stop (bool): If True, remove stopwords.
    remove_punct (bool): If True, remove punctuation.
    remove_email (bool): I

In [39]:
custom_preprocessor = cp.SpacyPreprocessor('en_core_web_sm')


In [40]:
def spacy_preprocessor(text):
    filtered_text = custom_preprocessor.transform([text])
    return " ".join(filtered_text)


In [41]:
# custom preprocessor and spacy tokenizer
vectorizer = CountVectorizer(analyzer='word', preprocessor=spacy_preprocessor,
                            token_pattern=r"[\S]+")
vectors = vectorizer.fit(Corpus)
vectorizer.vocabulary_


{'count': 2,
 'vectorizer': 11,
 'scoring': 9,
 'base': 0,
 'frequency': 4,
 'key': 6,
 'tfidf': 10,
 'high': 5,
 'score': 8,
 'binary': 1,
 'presence': 7,
 'word': 12,
 'dummy': 3}

#### <font color = 'pickle'>**token patterns with regular expressions**

In [42]:
# We can pass regex to the argument token_pattern to get required pattern
# whitespace tokenizer
# This can be very useful if we have allready cleaned the text
vectorizer = CountVectorizer(analyzer='word', token_pattern=r"[\S]+")

# Assign the encoded(transformed) vectors to a variable
vectors = vectorizer.fit_transform(Corpus)

vectorizer.vocabulary_


{'count': 9,
 'vectorizer': 28,
 '-': 3,
 'for': 13,
 'this': 27,
 'vectorizer,': 29,
 'scoring': 24,
 'is': 17,
 'done': 11,
 'based': 7,
 'on': 21,
 'frequency.': 15,
 'frequency': 14,
 'key.': 19,
 '@vectorizer': 6,
 '#frequency': 1,
 '@frequency,': 5,
 'doesn’t': 10,
 'tfidf': 25,
 'tfidf,': 26,
 'higher': 16,
 'score': 23,
 '#tfidf': 2,
 'binary': 8,
 'presence': 22,
 'of': 20,
 'word.': 30,
 'dummy': 12,
 'key': 18,
 '#dummy': 0,
 '@dummy': 4}

### <font color = 'pickle'>**ngrams**</font>

- Till now our features consists of single token. However, in some cases we may want to use sequence of tokens as features
- Consider the following corpus
 1. This item is good
 2. This item is not good
- Now  both the documents will have feature 'good' and 'not' will be an additional feature in document 2.
- For applications like sentiment analysis - it might be a good idea to consider 'not good' as a single token.

- We can use ngram_range(min_n, max_n) in CountVectorizer to create features that consists of sequence of words.

- if we specify min_n = 2 and max_n = 3, we will get bigrams and trigrams as features.

In [43]:
min_n = 2
max_n = 2

# only works if analyzer = 'word'
vectorizer1 = CountVectorizer(analyzer='word', ngram_range=(min_n, max_n))
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(min_n, max_n))

text1 = ["This item is not good"]
text2 = ["This item is terribly good"]

# Fit the vectorizer to text
vectorizer1.fit_transform(text1)
vectorizer2.fit_transform(text2)

features1 = vectorizer1.get_feature_names_out()
features2 = vectorizer2.get_feature_names_out()

print('Features for text 1\n')
for feature in features1:
    print(feature)

print(f'\nFeatures for text 2\n')
for feature in features2:
    print(feature)

Features for text 1

is not
item is
not good
this item

Features for text 2

is terribly
item is
terribly good
this item


## <font color = 'pickle'>**Example : IMDB Data set**

### <font color = 'pickle'>**Import Data**

In [44]:
basepath

'/Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6342_Applied_Natural_Language_Processing/'

In [45]:
# Use train.csv of IMDB movie review data (we downloaded this in the last lecture)
base_folder = Path(basepath)
data_folder = base_folder / '0_Data_Folder'
train_data = data_folder / 'train.csv'
test_data = data_folder /'test.csv'


In [46]:
# Reading data
train_df = pd.read_csv(train_data, index_col=0)
test_df = pd.read_csv(test_data, index_col=0)
print(f'Shape of Training data set is : {train_df.shape}')
print(f'Shape of Test data set is : {test_df.shape}')
print(f'\nTop five rows of Training data set:\n')
train_df.head()


Shape of Training data set is : (25000, 2)
Shape of Test data set is : (25000, 2)

Top five rows of Training data set:



,Reviews,Labels
0,Ever wanted to know just how much Hollywood co...,1
1,The movie itself was ok for the kids. But I go...,1
2,You could stage a version of Charles Dickens' ...,1
3,this was a fantastic episode. i saw a clip fro...,1
4,and laugh out loud funny in many scenes.<br />...,1


### <font color = 'pickle'>**Generating Vocab**</font>
- <font color = 'indianred'>**Vocab should be created only based on training dataset**</font>
- We will generate vocab using CountVectorizer
- <font color = 'indianred'>**Use fit_transform() on Training data set**.
- **Use only transform() on Test dataset**. This make sures that we generate vocab only based on training dataset.

In [47]:
# Initialize vectorizer
nltk_stop_words = nltk_stopwords.words('english')
bag_of_word = CountVectorizer(stop_words=nltk_stop_words)

# Fit on training data
bag_of_word.fit(train_df['Reviews'].values)


CountVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [48]:
# get feature names
features = bag_of_word.get_feature_names_out()
features

array(['00', '000', '0000000000001', ..., 'østbye', 'über', 'üvegtigris'],
      dtype=object)

In [49]:
# check the legth of the vocab
len(features)


74704

### <font color = 'pickle'>**Create vectors for reviews**

In [50]:
# Transform the training and test dataset
bow_vector_train = bag_of_word.transform(train_df['Reviews'].values)
bow_vector_test = bag_of_word.transform(test_df['Reviews'].values)


In [51]:
# Shape of the matrix for train dataset
bow_vector_train


<25000x74704 sparse matrix of type '<class 'numpy.int64'>'
	with 2479678 stored elements in Compressed Sparse Row format>

In [52]:
# Shape of the matrix for test dataset
bow_vector_test


<25000x74704 sparse matrix of type '<class 'numpy.int64'>'
	with 2385031 stored elements in Compressed Sparse Row format>

### <font color = 'pickle'>**Limit vocab using max_features**
We got 25k rows with 78k+ features, but what if we want only top 5k features.
We can do this by providing max_features parameter.

In [53]:
# Limit Vocab size using Max features
spacy_stop_words = nlp.Defaults.stop_words
bag_of_word = CountVectorizer(
    max_features=5000, stop_words=list(spacy_stop_words))  # Max features

# Fit on training data
bag_of_word.fit(train_df['Reviews'].values)


/opt/homebrew/anaconda3/envs/nlp/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(


CountVectorizer(max_features=5000,
                stop_words=['namely', 'now', 'empty', 'while', 'n’t', 'nowhere',
                            'hers', 'we', "'s", 'against', 'forty', 'used',
                            'among', 'he', 'otherwise', 'yourselves', 'except',
                            'them', 'should', 'across', 'full', 'because', 'ca',
                            'further', 'ever', 'under', 'became', 'by',
                            'thence', 'which', ...])

In [54]:
# Transform the training and test dataset
bow_vector_train = bag_of_word.transform(train_df['Reviews'].values)
bow_vector_test = bag_of_word.transform(train_df['Reviews'].values)


In [55]:
# Document representation
vocab = bag_of_word.get_feature_names_out()
pd.DataFrame(bow_vector_train.toarray(), columns=vocab)


,00,000,10,100,11,12,13,13th,14,15,...,yesterday,york,young,younger,youth,zero,zizek,zombie,zombies,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24997,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
24998,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
